In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

anos = range(2019, 2025)
dfs = [pd.read_csv(f'IMP_{ano}.csv', sep=';', dtype={'CO_NCM': str}) for ano in anos]
df = pd.concat(dfs, ignore_index=True)


In [ ]:
df = df[df['CO_NCM'] == '84713012'].copy()
df['CO_ANO'] = df['CO_ANO'].astype(int)
df['CO_MES'] = df['CO_MES'].astype(int)
df['date'] = pd.to_datetime(df['CO_ANO'].astype(str) + '-' + df['CO_MES'].astype(str).str.zfill(2) + '-01')
ts = df.groupby('date')['KG_LIQUIDO'].sum().sort_index()
ts = ts.asfreq('MS', fill_value=0)


In [ ]:
train = ts[:'2023-12-01']
test = ts['2024-01-01':]

In [ ]:

from pmdarima import auto_arima


model = auto_arima(train, seasonal=True, m=1, trace=True,
                   error_action='ignore', suppress_warnings=True)

forecast = model.predict(n_periods=len(test)) # Prever os 12 meses de 2024


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train.index, train, label='Treino')
plt.plot(test.index, test, label='Real 2024')
plt.plot(test.index, forecast, label='Previsão 2024', linestyle='--')
plt.xlabel('Data')
plt.ylabel('KG Líquido')
plt.title('Previsão ARIMA para 2024: Notebooks Pequenos ou Tablets (NCM 84713012)')
plt.legend()
plt.tight_layout()
plt.show()
